# Recommendation system with knn
- used_cols = beer_name, review_profilename, review_overall

In [33]:
import pandas as pd

path = '/Users/gwag-eunji/Downloads/beer_reviews.csv'
data = pd.read_csv(path)
data.head()
len(data.columns)

13

In [34]:
re_data = data.iloc[1:10001]
re_data.tail()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
9996,21590,Ram Restaurant & Brewery,1259451373,3.5,3.5,3.5,jdense,American IPA,3.5,3.0,Big Red IPA,6.0,54411
9997,5049,Bath Ales Ltd,1277680856,3.5,3.5,3.5,AEK,English Bitter,3.5,3.5,Golden Hare,4.4,52239
9998,5049,Bath Ales Ltd,1205875855,4.0,4.0,4.0,wl0307,English Bitter,3.5,4.0,SPA,3.7,41647
9999,5049,Bath Ales Ltd,1312801866,3.5,4.0,4.5,fullsweep,English Bitter,3.5,3.5,Wild Hare,5.0,34416
10000,5049,Bath Ales Ltd,1238938746,4.0,4.0,3.0,BlackHaddock,English Bitter,4.0,4.5,Wild Hare,5.0,34416


In [35]:
user_info = re_data.iloc[:,-7:-6]
beer_info = re_data.iloc[:, -3:-2]
rating_info = re_data.iloc[:, 3:6]
user_info
rating_info

,review_overall,review_aroma,review_appearance
1,3.0,2.5,3.0
2,3.0,2.5,3.0
3,3.0,3.0,3.5
4,4.0,4.5,4.0
5,3.0,3.5,3.5
...,...,...,...
9996,3.5,3.5,3.5
9997,3.5,3.5,3.5
9998,4.0,4.0,4.0
9999,3.5,4.0,4.5


In [36]:
concat_data = pd.concat([user_info, beer_info, rating_info], axis=1)
concat_data.head()

,review_profilename,beer_name,review_overall,review_aroma,review_appearance
1,stcules,Red Moon,3.0,2.5,3.0
2,stcules,Black Horse Black Beer,3.0,2.5,3.0
3,stcules,Sausa Pils,3.0,3.0,3.5
4,johnmichaelsen,Cauldron DIPA,4.0,4.5,4.0
5,oline73,Caldera Ginger Beer,3.0,3.5,3.5


In [42]:
user_rating = concat_data.drop_duplicates(['beer_name', 'review_profilename'])
user_rating.head()
user_rating_pivot = user_rating.pivot(index='beer_name', columns='review_profilename', values='review_overall').fillna(0)
user_rating_pivot.shape

(945, 3192)

In [43]:
from scipy.sparse import csr_matrix
user_rating_matrix = csr_matrix(user_rating_pivot.values)
user_rating_matrix
user_rating_pivot.shape

(945, 3192)

In [130]:
#KNN
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric="cosine",algorithm="brute")
model_knn.fit(user_rating_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [45]:
import numpy as np
query_index = np.random.choice(user_rating_pivot.shape[0])
print(user_rating_pivot.iloc[query_index,:])

review_profilename
NaN         0.0
05Harley    0.0
1759Girl    0.0
1Adam12     0.0
1fastz28    0.0
           ... 
zimm421     0.0
zoolzoo     0.0
zoso1967    0.0
zrab11      0.0
zuggy9      0.0
Name: Stella Export, Length: 3192, dtype: float64


In [46]:
distances, indices = model_knn.kneighbors(user_rating_pivot.iloc[query_index,:].values.reshape(1, -1)
                                         , n_neighbors=6)
print('Distances=',distances, 'Indices=', indices)
print(distances.flatten())
print(len(distances.flatten()))

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('recommendation for {0}:\n'.format(user_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distacne of {2}:'.format(i, user_rating_pivot.index[indices.flatten()[i]],
                                                      distances.flatten()[i]) )

Distances= [[0.         0.23177872 0.23177872 0.23177872 0.31134996 0.64981224]] Indices= [[796 944 740 797 552 553]]
[0.         0.23177872 0.23177872 0.23177872 0.31134996 0.64981224]
6
recommendation for Stella Export:

1: Zhigulovskoye, with distacne of 0.23177872040262415:
2: Sakara Red, with distacne of 0.23177872040262415:
3: Stella Premium, with distacne of 0.23177872040262415:
4: Meister, with distacne of 0.31134996124430514:
5: Meister Max, with distacne of 0.6498122405140894:


# prediction

In [52]:
user_rating_pivot.head()
user_rating_matrix = user_rating.pivot_table('review_overall', index='review_profilename', columns='beer_name')
user_rating_matrix = user_rating_matrix.fillna(0)
user_rating_matrix.head(3)

beer_name,'Pooya Porter,'Sconnie Pale Ale,'Sconnie Rustic Trail Amber,'Sconnie Tall Blonde Ale,'Tis The Seasonator,1 A.M. Ale,10 Blocks South,10 Year Clelebration Ale,12 Year Anniversary Ale,13th Anniversary Imperial India Pale Ale,...,X-Extra Pale Ale,Yakima Monster,Yankee Clipper IPA,Yellowjacket Ale,YouEnjoyMyStout,Zahre Affumicata,Zahre Canapa,Zahre Chiara Pilsen,Zahre Rossa Vienna,Zhigulovskoye
review_profilename,,,,,,,,,,,,,,,,,,,,,
05Harley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1759Girl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1Adam12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
user_rating_matrix_t = user_rating_matrix.transpose()
user_rating_matrix_t.head(3)

review_profilename,05Harley,1759Girl,1Adam12,1fastz28,2378GCGTG,23fyerfyter,31thadrty1,3Vandoo,4000qtrap,4DAloveofSTOUT,...,zeff80,zeledonia,zerk,zhanson88,ziggy13,zimm421,zoolzoo,zoso1967,zrab11,zuggy9
beer_name,,,,,,,,,,,,,,,,,,,,,
'Pooya Porter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Sconnie Pale Ale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Sconnie Rustic Trail Amber,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
print(user_rating_matrix.shape)
print(user_rating_matrix_t.shape)

(3191, 945)
(945, 3191)


In [58]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(user_rating_matrix_t, user_rating_matrix_t)

item_sim_df = pd.DataFrame(data=item_sim, 
                          index=user_rating_matrix.columns,
                          columns = user_rating_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head()

(945, 945)


beer_name,'Pooya Porter,'Sconnie Pale Ale,'Sconnie Rustic Trail Amber,'Sconnie Tall Blonde Ale,'Tis The Seasonator,1 A.M. Ale,10 Blocks South,10 Year Clelebration Ale,12 Year Anniversary Ale,13th Anniversary Imperial India Pale Ale,...,X-Extra Pale Ale,Yakima Monster,Yankee Clipper IPA,Yellowjacket Ale,YouEnjoyMyStout,Zahre Affumicata,Zahre Canapa,Zahre Chiara Pilsen,Zahre Rossa Vienna,Zhigulovskoye
beer_name,,,,,,,,,,,,,,,,,,,,,
'Pooya Porter,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Sconnie Pale Ale,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Sconnie Rustic Trail Amber,0.0,0.0,1.000000,0.894427,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Sconnie Tall Blonde Ale,0.0,0.0,0.894427,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Tis The Seasonator,0.0,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
item_sim_df["'Sconnie Tall Blonde Ale"].sort_values(ascending=False)[:6]

beer_name
'Sconnie Tall Blonde Ale       1.000000
'Sconnie Rustic Trail Amber    0.894427
SueBee Honey Ale               0.336994
'Pooya Porter                  0.000000
Orgemont Blonde                0.000000
Otro Mundo Nut Brown Ale       0.000000
Name: 'Sconnie Tall Blonde Ale, dtype: float64

In [65]:
def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [66]:
ratings_pred = predict_rating(user_rating_matrix.values , item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= user_rating_matrix.index,
                                   columns = user_rating_matrix.columns)
print(ratings_pred_matrix.shape)
ratings_pred_matrix.head(3)

(3191, 945)


beer_name,'Pooya Porter,'Sconnie Pale Ale,'Sconnie Rustic Trail Amber,'Sconnie Tall Blonde Ale,'Tis The Seasonator,1 A.M. Ale,10 Blocks South,10 Year Clelebration Ale,12 Year Anniversary Ale,13th Anniversary Imperial India Pale Ale,...,X-Extra Pale Ale,Yakima Monster,Yankee Clipper IPA,Yellowjacket Ale,YouEnjoyMyStout,Zahre Affumicata,Zahre Canapa,Zahre Chiara Pilsen,Zahre Rossa Vienna,Zhigulovskoye
review_profilename,,,,,,,,,,,,,,,,,,,,,
05Harley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0044,0.00000,0.0,...,0.0,0.0,0.000000,0.0,0.01549,0.0,0.0,0.0,0.0,0.0
1759Girl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.00000,0.0,...,0.0,0.0,0.012501,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
1Adam12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0044,0.02115,0.0,...,0.0,0.0,0.000000,0.0,0.01549,0.0,0.0,0.0,0.0,0.0


In [131]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 맥주에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, user_rating_matrix.values ))

아이템 기반 모든 인접 이웃 MSE:  9.903406835555096


In [79]:
query_index = np.random.choice(user_rating_pivot.shape[0])
print(query_index)
#print(user_rating_pivot.iloc[query_index,:])
#user_rating_matrix.loc[9, :] 
user_rating_id = user_rating_pivot.iloc[query_index,:]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

466


review_profilename
northyorksammy    2.5
Name: Irish Amber Light, dtype: float64

In [103]:
#마셔보지 않았던 것들을 unseen_list에 저장
already_seen = user_rating_id[ user_rating_id > 0].index.tolist()
already_seen
beer_list = user_rating_matrix.columns.tolist()
unseen_list = [beer for beer in beer_list if beer not in already_seen]
unseen_list

["'Pooya Porter",
 "'Sconnie Pale Ale",
 "'Sconnie Rustic Trail Amber",
 "'Sconnie Tall Blonde Ale",
 "'Tis The Seasonator",
 '1 A.M. Ale',
 '10 Blocks South',
 '10 Year Clelebration Ale',
 '12 Year Anniversary Ale',
 '13th Anniversary Imperial India Pale Ale',
 '14 Year Anniversary Ale',
 '15th Anniversary',
 '16 Year Anniversary Ale',
 '17 Year Anniversary Ale',
 '1871 Lager',
 '2 Brewers 1 Pint',
 '2001 - A Stout Odyssey',
 '60 Schilling Scottish Ale',
 '80 Weight Mild Amber Scotch Ale',
 "Ackerman's Imperial Double Stout (Indiana Replicale 2011)",
 "Ackerman's Imperial Double Stout (Winterfest Replicale 2011)",
 'Aguila Del Sur Rubia',
 'Akron Dopple Alt',
 "Al's Bock",
 'Albatross Lager',
 'Alcatraz Stout',
 'Aldersbacher Kloster Weisse Hell',
 'Alexander Gunn Barleywine',
 'Alpha Beta',
 'Alt',
 'Alt-Ernative Amber',
 'Altbier',
 'Amalgamated Dunkel',
 'Amalgamated Helles',
 'Amalgamated IPA',
 'Amalgamated Märzen',
 'Amalgamated Rauchbier',
 'Amalgamated Zoigl',
 'Amarillo Lager

In [114]:
ratings_pred_matrix

beer_name,'Pooya Porter,'Sconnie Pale Ale,'Sconnie Rustic Trail Amber,'Sconnie Tall Blonde Ale,'Tis The Seasonator,1 A.M. Ale,10 Blocks South,10 Year Clelebration Ale,12 Year Anniversary Ale,13th Anniversary Imperial India Pale Ale,...,X-Extra Pale Ale,Yakima Monster,Yankee Clipper IPA,Yellowjacket Ale,YouEnjoyMyStout,Zahre Affumicata,Zahre Canapa,Zahre Chiara Pilsen,Zahre Rossa Vienna,Zhigulovskoye
review_profilename,,,,,,,,,,,,,,,,,,,,,
05Harley,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.004400,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.015490,0.0,0.0,0.0,0.0,0.0
1759Girl,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.012501,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1Adam12,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.004400,0.021150,0.000000,...,0.0,0.0,0.000000,0.0,0.015490,0.0,0.0,0.0,0.0,0.0
1fastz28,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.011692,0.0,0.0,0.0,0.0,0.0
2378GCGTG,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.029610,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zimm421,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.002883,0.000000,0.000000,...,0.0,0.0,0.005662,0.0,0.009022,0.0,0.0,0.0,0.0,0.0
zoolzoo,0.007065,0.000000,0.0,0.0,0.0,0.00000,0.0,0.003511,0.014555,0.030272,...,0.0,0.0,0.006703,0.0,0.009612,0.0,0.0,0.0,0.0,0.0
zoso1967,0.007948,0.000000,0.0,0.0,0.0,0.00000,0.0,0.014796,0.016374,0.034056,...,0.0,0.0,0.083574,0.0,0.066591,0.0,0.0,0.0,0.0,0.0


In [118]:
recommen_beers = ratings_pred_matrix.iloc[query_index,:].sort_values(ascending=False)[:10]

In [119]:
recommen_beers

beer_name
Organic Amber                  0.151033
American Pale Ale              0.100796
Suicide By Hops                0.095591
Terminator Stout               0.092953
India Pale Ale                 0.070181
Brau Rauck                     0.064148
Oatboy Imperial IPA            0.063672
Bourbon Barrel Beer De Mars    0.058442
Riley's Rye                    0.054595
Caldera OBF 15                 0.053395
Name: Abita45, dtype: float64

# SGD

In [120]:
def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [122]:
#행렬 분해 함수 
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트. 
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
                       # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

In [123]:
P, Q = matrix_factorization(user_rating_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse :  3.7622061117875494
### iteration step :  10  rmse :  1.5538161532657306
### iteration step :  20  rmse :  0.9463720079640513
### iteration step :  30  rmse :  0.641505708406177
### iteration step :  40  rmse :  0.45879865176996204
### iteration step :  50  rmse :  0.34626060878574394
### iteration step :  60  rmse :  0.26704254710547326
### iteration step :  70  rmse :  0.20494221782344568
### iteration step :  80  rmse :  0.15680866085988246
### iteration step :  90  rmse :  0.1224906064592438
### iteration step :  100  rmse :  0.09925587193995002
### iteration step :  110  rmse :  0.08347855504132873
### iteration step :  120  rmse :  0.07244905896188215
### iteration step :  130  rmse :  0.06441716516670971
### iteration step :  140  rmse :  0.05832594700582296
### iteration step :  150  rmse :  0.05357187775058327
### iteration step :  160  rmse :  0.04980468343290139
### iteration step :  170  rmse :  0.04679774086562891
### iteration step :  180  

In [126]:
sgd_ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= user_rating_matrix.index,
                                   columns = user_rating_matrix.columns)

sgd_ratings_pred_matrix.head(3)

beer_name,'Pooya Porter,'Sconnie Pale Ale,'Sconnie Rustic Trail Amber,'Sconnie Tall Blonde Ale,'Tis The Seasonator,1 A.M. Ale,10 Blocks South,10 Year Clelebration Ale,12 Year Anniversary Ale,13th Anniversary Imperial India Pale Ale,...,X-Extra Pale Ale,Yakima Monster,Yankee Clipper IPA,Yellowjacket Ale,YouEnjoyMyStout,Zahre Affumicata,Zahre Canapa,Zahre Chiara Pilsen,Zahre Rossa Vienna,Zhigulovskoye
review_profilename,,,,,,,,,,,,,,,,,,,,,
05Harley,1.597182,1.573955,1.326180,0.948194,1.034236,1.488021,1.849904,0.781357,1.139725,1.560191,...,1.891806,1.245735,0.975085,1.349652,1.447955,1.233700,0.800898,0.957803,0.183973,1.477247
1759Girl,1.132585,1.438912,1.407245,0.973328,0.680857,1.421659,1.504296,0.610312,0.964989,1.244879,...,1.916373,1.138112,0.836298,1.090443,1.348660,1.052220,0.720236,0.811964,-0.168036,1.293739
1Adam12,1.539012,1.538969,1.258291,0.905066,1.051479,1.430300,1.830801,0.728642,1.084829,1.508709,...,1.796906,1.188891,0.950946,1.347445,1.465810,1.190922,0.770849,0.931019,0.181743,1.465572


In [128]:
sgd_recommen_beers = sgd_ratings_pred_matrix.iloc[query_index,:].sort_values(ascending=False)[:10]

In [129]:
sgd_recommen_beers

beer_name
American Pale Ale                                 1.980108
B.O.R.I.S. The Crusher Oatmeal-Imperial Stout     1.858585
BORIS Royale                                      1.756666
The Worthy Adversary                              1.752067
War And Peace                                     1.742536
Schwarz Beer                                      1.736191
Barrel Aged Naked Evil                            1.724208
Mogli                                             1.719476
Barrel Aged B.O.R.I.S. Oatmeal Imperial Stout     1.701107
D.O.R.I.S. The Destroyer Double Imperial Stout    1.695070
Name: Abita45, dtype: float64